# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
# Study data files
weather_file = "../output_data/Exported_weather_data.csv"

# Read the mouse data and the study results
weather_data = pd.read_csv(weather_file)
weather_data=weather_data.drop('Unnamed: 0', axis=1)
weather_data.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Rikitea,-23.12,-134.97,20.80,80,12,9.84,PF,1595703320
1,Port Augusta,-32.50,137.77,9.00,93,90,1.50,AU,1595703423
2,São Miguel do Araguaia,-13.28,-50.16,33.23,25,0,1.87,BR,1595703426
3,Alofi,-19.06,-169.92,22.00,83,18,4.10,NU,1595703429
4,Lebu,-37.62,-73.65,11.09,83,100,9.58,CL,1595703323


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
ideal=weather_data.loc[(weather_data['Max_Temp']<30)]
ideal=ideal.loc[(ideal['Max_Temp']>20)]
ideal=ideal.loc[(ideal['Wind_Speed']<5)]
ideal=ideal.loc[(ideal['Humidity']<50)]
ideal.count()


City          24
Lat           24
Lng           24
Max_Temp      24
Humidity      24
Cloudiness    24
Wind_Speed    24
Country       23
Date          24
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df=ideal
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
13,Kumul,42.80,93.45,28.97,20,0,3.91,CN,1595703464,
58,East London,-33.02,27.91,21.95,27,0,3.88,ZA,1595703632,
81,Laguna,38.42,-121.42,27.78,41,5,1.50,US,1595703715,
86,Lüderitz,-26.65,15.16,23.00,27,0,2.10,NaN,1595703740,
97,Port Alfred,-33.59,26.89,20.94,31,0,4.90,ZA,1595703778,


In [50]:
hotels=[]
count=0
# hotel=row['Hotel Name']
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    

    
    target=(f'{lat} , {lng}')
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

# set up a parameters dictionary
    params = {
        "location": target,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

    response = requests.get(base_url, params=params)
    places_data=response.json()
    try:
        hotel_df.loc[index,'Hotel Name']=places_data["results"][0]["name"]
#         hotels.append(places_data["results"][0]["name"])
#         hotel.append(places_data["results"][0]["name"])
        count+=1
        print(f'Found hotel for location {count}')
        
    except:
        count+=1
        print(f'No hotel for location {count}')
# for i in range(0,len(hotel_df['City'])):
#     print(places_data["results"][i]["name"])
#     print(places_data["results"][i]["vicinity"])

Found hotel for location 1
Found hotel for location 2
Found hotel for location 3
Found hotel for location 4
Found hotel for location 5
Found hotel for location 6
Found hotel for location 7
No hotel for location 8
No hotel for location 9
Found hotel for location 10
Found hotel for location 11
Found hotel for location 12
No hotel for location 13
Found hotel for location 14
Found hotel for location 15
Found hotel for location 16
Found hotel for location 17
Found hotel for location 18
Found hotel for location 19
Found hotel for location 20
Found hotel for location 21
Found hotel for location 22
No hotel for location 23
Found hotel for location 24


In [51]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
13,Kumul,42.80,93.45,28.97,20,0,3.91,CN,1595703464,7 Days Inn
58,East London,-33.02,27.91,21.95,27,0,3.88,ZA,1595703632,Stirling
81,Laguna,38.42,-121.42,27.78,41,5,1.50,US,1595703715,Hilton Garden Inn Sacramento Elk Grove
86,Lüderitz,-26.65,15.16,23.00,27,0,2.10,NaN,1595703740,LÜDERITZ NEST HOTEL
97,Port Alfred,-33.59,26.89,20.94,31,0,4.90,ZA,1595703778,The Halyards Hotel
234,Taltal,-25.40,-70.48,20.98,26,0,2.02,CL,1595704270,HOTEL PLAZA
258,Agía Galíni,35.10,24.68,28.33,37,0,3.34,GR,1595704349,Palazzo Greco
288,Sangīn,32.07,64.84,29.28,14,0,0.88,AF,1595704447,
305,Yumen,40.28,97.20,22.71,21,0,4.66,CN,1595704310,
309,Kasongo,-4.45,26.67,26.16,30,35,2.09,CD,1595704516,Kago Hotel


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))